## Prerequisites

In [ ]:
!pip install pandas
!pip install sentencepiece
!pip install hgtk
!pip install gluonnlp

     |████████████████████████████████| 1.1MB 1.1MB/s 
  Created wheel for hgtk: filename=hgtk-0.1.3-py2.py3-none-any.whl size=6688 sha256=90829c1abb57e4fece62d374291acbe8ac91b59ffeca4a3e6a61dcfc0a4d9072
  Stored in directory: /root/.cache/pip/wheels/73/72/06/6065a57fe68264f35d7e52e37f56831eb3e9ec75656880de20
Successfully built hgtk
     |████████████████████████████████| 348kB 2.7MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588518 sha256=442ed09c75a6f7d6925401908027703ee2347a2f6be2a9e3a18a70cd07a930ba
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp


## Base Function

In [ ]:
def file_num_padding(file_num) :
    if file_num < 10 :
        return '00000' + str(file_num)
    elif file_num < 100 :
        return '0000' + str(file_num)
    elif file_num < 1000 :
        return '000' + str(file_num)
    elif file_num < 10000 :
        return '00' + str(file_num)
    elif file_num < 100000 :
        return '0' + str(file_num)
    else :
        return str(file_num)

In [ ]:
def get_path(path, fname, file_num, format):
    return path + fname +file_num + format

In [ ]:
BASE_PATH = './content/drive/My Drive/googledrive/'
fname = 'KsponSpeech_'
file_num = 1
format = '.txt'

print(get_path(BASE_PATH, fname, file_num_padding(file_num), format))

./content/drive/My Drive/googledrive/KsponSpeech_000001.txt


## preprocess

In [ ]:
import os
import re

In [1]:
# mode = spelling(e.g., 70%는 칠십퍼센트 이렇게 표기되는 게 아니라 70%로 표기되도록)
def bracket_filter(sentence):
    # new_sentence = str()
    new_sentence = ""
    # 디폴트 값은 False
    flag = False

    for ch in sentence :
        if ch == '(' :
            if flag == False:
                flag = True
            continue

        if ch == ')' :
            if flag == True :
                flag = False
                continue

        if ch != ')' and flag == False :
            new_sentence += ch

        return new_sentence

In [ ]:
# 되는지 테스트 해보기

## Special Filter

In [ ]:
def special_filter(sentence):
    SENTENCE_MARK = ['?', '!', '.', ',']
    # noise라고 인식하면 안 되는 것들 추려내기!
    # b = 숨소리, l = 웃음 소리, o = 다른 사람의 말소리가 포함된 경우 문장의 맨 앞에 표기, n = 주변의 잡음
    NOT_NOISE = ['b', 'l']
    # u = 문맥을 보아도 도저히 알 수 없는 발음
    NOISE = ['o', 'n', 'u']
    EXCEPT = ['/', '+', '*', '-', '@', '$', '^', '&', '[', ']', '=', ':', ';', ',']

    new_sentence = ""
    for idx, ch in enumerate(sentence):
        if ch not in SENTENCE_MARK:
            if idx + 1 < len(sentence) and ch in NOISE and sentence[idx + 1] == '/':
                continue

        # 여기에다가 not_noise 부분 처리해야 되나?
        elif ch not in EXCEPT:
            new_sentence += ch
    pattern = re.compile(r'\s\s+')
    new_sentence = re.sub(pattern, ' ', new_sentence.strip())
    return new_sentence